In [1]:
from pyspark.sql import SparkSession

In [2]:
from delta import *

In [3]:
builder = SparkSession.builder.appName("MyApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/10/04 21:49:33 WARN Utils: Your hostname, deepak-ubuntu, resolves to a loopback address: 127.0.1.1; using 192.168.31.105 instead (on interface wlp1s0)
25/10/04 21:49:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/deepak/anaconda3/envs/ai_data/lib/python3.13/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/deepak/.ivy2.5.2/cache
The jars for the packages stored in: /home/deepak/.ivy2.5.2/jars
io.delta#delta-spark_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-1d3cc87f-b67c-4065-8cab-29a199cbe623;1.0
	confs: [default]
	found io.delta#delta-spark_2.13;4.0.0 in central
	found io.delta#delta-storage;4.0.0 in central
	found org.antlr#antlr4-runtime;4.13.1 in central
:: resolution report :: resolve 402ms :: artifacts dl 8ms
	::

In [4]:
spark.sql("""
CREATE OR REPLACE TABLE people USING DELTA
AS SELECT * FROM VALUES
  (1, 'Alice', 10),
  (2, 'Bob', 20),
  (3, 'Charlie', 30)
  AS t(id, name, age)
""")

25/10/04 21:49:53 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
25/10/04 21:49:58 WARN OptimisticTransaction: Change in the table id detected in txn. Table id for txn on table at file:/home/deepak/programs/python/sparkLearn/DifferentSources/spark-warehouse/people was 6dd47f0e-29f9-429f-abc1-1c80be839d79 when the txn was created and is now changed to 5e1f28cc-b03e-4f80-95e1-7c42245aeead.
25/10/04 21:49:59 WARN DeltaLog: Change in the table id detected while updating snapshot. 
Previous snapshot = Snapshot(path=file:/home/deepak/programs/python/sparkLearn/DifferentSources/spark-warehouse/people/_delta_log, version=1, metadata=Metadata(6dd47f0e-29f9-429f-abc1-1c80be839d79,null,null,Format(parquet,Map()),{"type":"struct","fields":[{"name":"id","type":"integer","nullable":true,"metadata":{}},{"name":"name","type":"string","nullable":true,"metadata":{}},{"name":"age","

DataFrame[]

In [5]:
spark.sql("DESCRIBE DETAIL people").show(truncate=False)


+------+------------------------------------+----------------------------+-----------+------------------------------------------------------------------------------------+-----------------------+-----------------------+----------------+-----------------+--------+-----------+----------+----------------+----------------+------------------------+
|format|id                                  |name                        |description|location                                                                            |createdAt              |lastModified           |partitionColumns|clusteringColumns|numFiles|sizeInBytes|properties|minReaderVersion|minWriterVersion|tableFeatures           |
+------+------------------------------------+----------------------------+-----------+------------------------------------------------------------------------------------+-----------------------+-----------------------+----------------+-----------------+--------+-----------+----------+----------------+-----

In [6]:
spark.sql("""
Insert into people
          values 
          (4, 'Deepak', 24),
          (5, 'Srikanth', 27)
""")

DataFrame[]

In [7]:
spark.sql("DESCRIBE HISTORY people").show(truncate=False)

+-------+-----------------------+------+--------+---------------------------------+----------------------------------------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+----------------------------------------------------------------------------------------------------------+------------+-----------------------------------+
|version|timestamp              |userId|userName|operation                        |operationParameters                                                                           |job |notebook|clusterId|readVersion|isolationLevel|isBlindAppend|operationMetrics                                                                                          |userMetadata|engineInfo                         |
+-------+-----------------------+------+--------+---------------------------------+----------------------------------------------------------------------------------------------+----+--------+--------

In [8]:
spark.sql("""
select * from people version as of 1
""").orderBy('id').show()

+---+--------+---+
| id|    name|age|
+---+--------+---+
|  1|   Alice| 10|
|  2|     Bob| 20|
|  3| Charlie| 30|
|  4|  Deepak| 24|
|  5|Srikanth| 27|
+---+--------+---+



In [9]:
spark.sql("""
show schemas
""").show()

+---------+
|namespace|
+---------+
|  default|
+---------+



In [10]:
spark.sql("""
show catalogs
""").show()

+-------------+
|      catalog|
+-------------+
|spark_catalog|
+-------------+



In [14]:
spark.sql("""
CREATE DATABASE gizmobox;
""")

DataFrame[]

In [13]:
spark.sql("""
drop schema gizmobox;
""")

DataFrame[]

In [15]:
spark.sql("""
show databases;
""").show()

+---------+
|namespace|
+---------+
|  default|
| gizmobox|
+---------+



In [18]:
spark.conf.get('spark.sql.warehouse.dir')

'file:/home/deepak/programs/python/sparkLearn/DifferentSources/spark-warehouse'

In [31]:
spark.stop()